<a href="https://colab.research.google.com/github/victorlf4/MineRLDecisionTransformers/blob/master/notebooks/Decision_Transformer_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
import plotly.io as pio

# Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
pio.renderers.default = "colab"

In [ ]:
!pip install git+https://github.com/neelnanda-io/Easy-Transformer.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/neelnanda-io/Easy-Transformer.git to /tmp/pip-req-build-j8hkwkal
  Running command git clone -q https://github.com/neelnanda-io/Easy-Transformer.git /tmp/pip-req-build-j8hkwkal
     |████████████████████████████████| 41 kB 458 kB/s 
     |████████████████████████████████| 441 kB 19.6 MB/s 
     |████████████████████████████████| 5.5 MB 33.0 MB/s 
     |████████████████████████████████| 1.9 MB 47.9 MB/s 
     |████████████████████████████████| 237 kB 66.3 MB/s 
     |████████████████████████████████| 212 kB 60.8 MB/s 
     |████████████████████████████████| 115 kB 75.5 MB/s 
     |████████████████████████████████| 95 kB 4.4 MB/s 
     |████████████████████████████████| 163 kB 74.6 MB/s 
     |████████████████████████████████| 127 kB 60.7 MB/s 
     |████████████████████████████████| 115 kB 70.7 MB/s 
     |████████████████████████████████| 51 kB 7.0 MB/s 
     

In [ ]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
import tqdm.notebook as tqdm

import random
import time

# from google.colab import drive
from pathlib import Path
import pickle
import os


import matplotlib.pyplot as plt

%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go

from torch.utils.data import DataLoader

from functools import *
import pandas as pd
import gc
import collections
import copy

# import comet_ml
import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets

import gym

In [ ]:
from easy_transformer.utils import (
  gelu_new,
  to_numpy,
  get_corner,
  lm_cross_entropy_loss,
)  # Helper functions
from easy_transformer.hook_points import (
  HookedRootModule,
  HookPoint,
)  # Hooking utilities
from easy_transformer import EasyTransformer, EasyTransformerConfig
import easy_transformer
from easy_transformer.experiments import (
  ExperimentMetric,
  AblationConfig,
  EasyAblation,
  EasyPatching,
  PatchingConfig,
)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
print(device)

cuda


Some plotting code. Wrappers around Plotly, not important to understand.

In [ ]:
def imshow(tensor, yaxis="", xaxis="", **kwargs):
  tensor = to_numpy(tensor)
  plot_kwargs = {"color_continuous_scale":"RdBu", "color_continuous_midpoint":0.0, "labels":{"x":xaxis, "y":yaxis}}
  plot_kwargs.update(kwargs)
  px.imshow(tensor, **plot_kwargs).show()

In [ ]:
def gymEnvSample(env, batch_size=5, sequence_size=10):
  batch = []

  for i in range(batch_size):
    env.reset()

    steps = []
    for j in range(sequence_size):
      action = env.action_space.sample()
      obs, reward, done, info = env.step(action)
      steps.append([reward, obs, action])

    batch.append(steps)

  tensor = torch.as_tensor(batch)
  return tensor.flatten(start_dim=1)

# Multi-armed bandits environment

In [ ]:
class MultiarmedBanditsEnv(gym.Env):
  """Environment for multiarmed bandits"""
  metadata = {'render.modes': ['human']}

  def __init__(self, nr_arms=10):
    super(MultiarmedBanditsEnv, self).__init__()
    self.action_space = gym.spaces.Discrete(nr_arms)
    self.observation_space = gym.spaces.Discrete(1)
    self.state = 0
    self.reset()

  def step(self, action):
    assert self.action_space.contains(action)
    reward = self.values[action]

    return self.state, reward, False, {self.optimal}

  def reset(self):
    self.values = list(range(self.action_space.n))
    np.random.shuffle(self.values)
    self.optimal = np.argmax(self.values)
    return self.state

  def render(self, mode='human', close=False):
    print("You are playing a %d-armed bandit" % self.action_space.n)

In [ ]:
gymEnvSample(MultiarmedBanditsEnv())

tensor([[4, 0, 8, 6, 0, 3, 2, 0, 7, 9, 0, 0, 9, 0, 0, 9, 0, 0, 7, 0, 4, 2, 0, 7,
         7, 0, 4, 0, 0, 2],
        [5, 0, 0, 6, 0, 1, 7, 0, 3, 4, 0, 2, 4, 0, 2, 1, 0, 7, 0, 0, 4, 6, 0, 1,
         0, 0, 4, 8, 0, 9],
        [6, 0, 1, 4, 0, 2, 6, 0, 1, 0, 0, 9, 4, 0, 2, 2, 0, 3, 0, 0, 9, 5, 0, 6,
         2, 0, 3, 3, 0, 4],
        [0, 0, 1, 5, 0, 0, 9, 0, 9, 0, 0, 1, 8, 0, 8, 6, 0, 3, 3, 0, 5, 9, 0, 9,
         1, 0, 2, 5, 0, 0],
        [5, 0, 3, 8, 0, 7, 4, 0, 8, 2, 0, 4, 2, 0, 4, 3, 0, 0, 6, 0, 1, 6, 0, 1,
         2, 0, 4, 9, 0, 2]])

# Treasure hunt environment

In [ ]:
class TreasureHuntEnv(gym.Env):
  """Environment for treasure hunt"""
  metadata = {'render.modes': ['human']}

  def __init__(self, nr_chests=10):
    super(TreasureHuntEnv, self).__init__()
    self.action_space = gym.spaces.Discrete(nr_chests)
    self.observation_space = gym.spaces.Discrete(1)
    self.state = 0
    self.reset()

  def step(self, action):
    assert self.action_space.contains(action)
    reward = self.values[action]

    return self.state, reward, False, {self.optimal}

  def reset(self):
    self.values = np.zeros(self.action_space.n, dtype=np.int32)
    self.values[np.random.randint(0, self.action_space.n)] = 1
    self.optimal = np.argmax(self.values)
    return self.state

  def render(self, mode='human', close=False):
    print("You are playing treasure hunt")

In [ ]:
gymEnvSample(TreasureHuntEnv())

tensor([[0, 0, 9, 0, 0, 3, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0, 0, 6, 0, 0, 4, 1, 0, 7,
         0, 0, 4, 0, 0, 5],
        [0, 0, 1, 0, 0, 1, 0, 0, 4, 0, 0, 1, 0, 0, 9, 0, 0, 0, 0, 0, 3, 0, 0, 3,
         0, 0, 6, 0, 0, 9],
        [0, 0, 7, 0, 0, 2, 0, 0, 0, 0, 0, 4, 0, 0, 6, 1, 0, 5, 0, 0, 4, 0, 0, 1,
         0, 0, 9, 0, 0, 0],
        [1, 0, 8, 0, 0, 0, 0, 0, 1, 0, 0, 7, 0, 0, 9, 0, 0, 9, 0, 0, 2, 0, 0, 0,
         0, 0, 7, 0, 0, 0],
        [0, 0, 6, 0, 0, 0, 0, 0, 9, 0, 0, 2, 0, 0, 9, 0, 0, 2, 0, 0, 7, 0, 0, 1,
         0, 0, 3, 0, 0, 2]])

# TestEnviroment

In [ ]:
class TestEnv(gym.Env):
  """Environment for treasure hunt"""
  metadata = {'render.modes': ['human']}

  def __init__(self, nr_chests=10):
    super(TestEnv, self).__init__()
    self.action_space = gym.spaces.Discrete(10)
    self.observation_space = gym.spaces.Discrete(7)
    self.state = 7
    self.reset()

  def step(self, action):
    assert self.action_space.contains(action)
    reward = action

    return self.state, reward, False,[0]

  def reset(self):
    return self.state

  def render(self, mode='human', close=False):
    print("You are playing treasure hunt")

In [ ]:
gymEnvSample(TestEnv())


tensor([[2, 7, 2, 3, 7, 3, 7, 7, 7, 7, 7, 7, 3, 7, 3, 9, 7, 9, 6, 7, 6, 3, 7, 3,
         1, 7, 1, 7, 7, 7],
        [0, 7, 0, 4, 7, 4, 6, 7, 6, 6, 7, 6, 0, 7, 0, 0, 7, 0, 0, 7, 0, 3, 7, 3,
         5, 7, 5, 5, 7, 5],
        [7, 7, 7, 2, 7, 2, 3, 7, 3, 9, 7, 9, 5, 7, 5, 8, 7, 8, 8, 7, 8, 6, 7, 6,
         3, 7, 3, 5, 7, 5],
        [5, 7, 5, 9, 7, 9, 1, 7, 1, 8, 7, 8, 2, 7, 2, 3, 7, 3, 8, 7, 8, 1, 7, 1,
         2, 7, 2, 1, 7, 1],
        [1, 7, 1, 1, 7, 1, 8, 7, 8, 9, 7, 9, 9, 7, 9, 2, 7, 2, 9, 7, 9, 9, 7, 9,
         1, 7, 1, 2, 7, 2]])

# Training the model

In [ ]:
tiny_cfg = EasyTransformerConfig(
  d_model=32,
  d_head=16,
  n_heads=2,
  d_mlp=128,
  n_layers=2,
  n_ctx=60,
  act_fn="solu_ln",
  d_vocab=10,
  normalization_type="LN",
  seed=23,  # Now we're training a custom model, it's good to set the seed to get reproducible results. It defaults to 42.
)

tiny_model = EasyTransformer(tiny_cfg).to(device)
tiny_optimizer = torch.optim.Adam(tiny_model.parameters(), lr=1e-3)
batch_size = 128
num_epochs = 301
env = TreasureHuntEnv()

assert tiny_model.cfg.n_ctx % 3 == 0, "n_ctx must be divisible by 3"

Moving model to device:  cuda
Moving model to device:  cuda


In [ ]:
losses = []

# TODO maybe use a learning rate scheduler
for epoch in tqdm.tqdm(range(num_epochs)):
  batch = gymEnvSample(env, batch_size=batch_size, sequence_size=int(tiny_model.cfg.n_ctx / 3))
  loss = tiny_model(batch, return_type="loss")
  loss.backward()
  tiny_optimizer.step()
  tiny_optimizer.zero_grad()
  losses.append(loss.item())
  if epoch % 100 == 0:
    print(f"Epoch: {epoch}. Loss: {loss}")

px.line(losses, labels={"x": "Epoch", "y": "Loss"})

  0%|          | 0/301 [00:00<?, ?it/s]

Epoch: 0. Loss: 2.5115251541137695
Epoch: 100. Loss: 1.17474365234375
Epoch: 200. Loss: 0.7520882487297058
Epoch: 300. Loss: 0.7476818561553955


In [ ]:
logits, tiny_cache = tiny_model.run_with_cache(batch)
print("Loss:", losses[-1])
print("Cache:", tiny_cache)

Loss: 0.7476818561553955
Cache: ActivationCache with keys ['hook_embed', 'hook_pos_embed', 'blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_attn', 'blocks.0.attn.hook_z', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_mid', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_mid', 'blocks.0.mlp.ln.hook_scale', 'blocks.0.mlp.ln.hook_normalized', 'blocks.0.mlp.hook_post', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.ln1.hook_scale', 'blocks.1.ln1.hook_normalized', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_attn', 'blocks.1.attn.hook_z', 'blocks.1.hook_attn_out', 'blocks.1.hook_resid_mid', 'blocks.1.ln2.hook_scale', 'blocks.1.ln2.hook_normalized', 'bloc

In [ ]:
with torch.no_grad():
  thing = torch.as_tensor([0, 0, 1])
  logit = tiny_model(thing)
  print(thing.shape, np.argmax(logit.cpu().numpy(), axis=2).shape)
  print(logit.shape, np.argmax(logit.cpu().numpy(), axis=2).shape)
  print(np.argmax(logit.cpu().numpy(), axis=2))
#for i in tqdm.tqdm(range(100)):
#    test_tokens = reference_gpt2.to_tokens(test_string).cuda()
#   demo_logits = demo_gpt2(test_tokens)
#    test_string += reference_gpt2.tokenizer.decode(demo_logits[-1, -1].argmax())
#print(test_string)

torch.Size([3]) (1, 3)
torch.Size([1, 3, 10]) (1, 3)
[[7 0 0]]


# Visualising attention patterns

In [ ]:
for layer in range(tiny_model.cfg.n_layers):
  for head in range(tiny_model.cfg.n_heads):
    imshow(to_numpy(tiny_cache[f'blocks.{layer}.attn.hook_attn'].mean(0)[head]), title=f'Layer {layer} Attention Pattern, Head {head}', height=500, width=500, xaxis="Token", yaxis="Activation")

/usr/local/lib/python3.7/dist-packages/plotly/express/_imshow.py:354: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [ ]:
# Install an older version of node, so that Svelte works (a web dev framework)
!curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
!pip install git+https://github.com/neelnanda-io/PySvelte.git
import sys
sys.path.append('/content/PySvelte')


## Installing the NodeSource Node.js 16.x repo...


## Populating apt-get cache...

+ apt-get update
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 https://deb.nodesource.com/node_16.x bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  I

In [ ]:
def array2text(array):
  str_array=[]
  assert len(array)% 3 == 0, "array lenght must be divisible by 3"
  for i in range(int(len(array)/3)):
      str_array.append("S"+str(i)+"_R:"+str(array[i]))
      str_array.append("S"+str(i)+"_O:"+str(array[i+1]))
      str_array.append("S"+str(i)+"_A:"+str(array[i+1]))
  return str_array

print(array2text([1,2,3]))



['S0_R:1', 'S0_O:2', 'S0_A:2']


In [ ]:
import pysvelte
tokens_test=[1,0,1,0,5,0,1,0,0]
test_tokens_str=array2text(tokens_test)
test_tokens=torch.as_tensor(tokens_test)
logits, tiny_cache = tiny_model.run_with_cache(test_tokens)

for layer in range(tiny_model.cfg.n_layers):
    print("Attention for layer", layer)
    
    print(tiny_cache[f'blocks.{layer}.attn.hook_attn'][0].shape)
    pysvelte.AttentionMulti(attention=tiny_cache[f'blocks.{layer}.attn.hook_attn'][0].permute(1, 2, 0),tokens=test_tokens_str).show() #.attn.hook_attn']

Attention for layer 0
torch.Size([2, 9, 9])
pysvelte components appear to be unbuilt or stale
Running npm install...
Building pysvelte components with webpack...


Attention for layer 1
torch.Size([2, 9, 9])


#Direct logit atribution
##Neels explanation

The residual stream is the sum of the output of each layer, so we can map each layer output to a set of logits, and look at the contribution to the logit diff to figure out how much it contributes.

Further, as the output of the attn layer is the sum of each head (plus a bias term), we can look at each head's contribution.

This is called the direct logit attribution - direct since we're looking at the logits, so there's just a linear map (holding layernorm scale fixed) (Exercise: Why is this not direct if we're looking at the loss or log probs?)

We can calculate this more efficiently by dotting with W_U[John] - W_U[Mary], which just a direction in the residual stream

In [ ]:
#TODO modify this to work on our code

# Set use attn result to True - this gives us a hook for the result of each head, 
# ie the d_model length vectors whose sum makes up attn_out and is added to the residual stream
model.cfg.use_attn_result = True

logit_diff_direction = model.unembed.W_U[:, john_index] - model.unembed.W_U[:, mary_index]
# Take the scaling factor of the layernorm pre-unembed on the final token, so our logit attrs are on the same scale
final_layer_norm_scale = (tiny_cache['ln_final.hook_scale'][0, -1])
print("Final layer norm scaling factor:", final_layer_norm_scale.item())

direct_logit_attr = torch.zeros(model.cfg.n_layers, model.cfg.n_heads).to(device)
def calc_direct_logit_attr(result, hook):
    layer = int(hook.name.split('.')[1])
    final_token_result = result[0, -1]
    direct_logit_attr[layer] = (final_token_result @ (logit_diff_direction))/final_layer_norm_scale

model.run_with_hooks(example_text, fwd_hooks = [(lambda name:name.endswith('hook_result'), calc_direct_logit_attr)])

imshow(direct_logit_attr, xaxis='Head', yaxis='Layer', title='Direct Logit Attribution')

# Switch use_attn_result back off, since it consumes a lot of memory (Exercise: Why is this more expensive than eg calculating the value?)
model.cfg.use_attn_result = False

# Quincallada

In [ ]:
imshow(to_numpy(tiny_cache["attn", 0].mean([0, 1])), title="Layer 0 Attention Pattern", height=500, width=500)

/usr/local/lib/python3.7/dist-packages/plotly/express/_imshow.py:354: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [ ]:
env = TreasureHuntEnv()
for i in range(2000):
  action = int(input("Action: "))
  obs, rewards, done, info = env.step(action)
  print(rewards)

KeyboardInterrupt: ignored